# 6.2 Subsets and slices of ordered pairs

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In many applications, we are concerned only with a subset of all ordered pairs from
two sets. For example, in the transportation model, shipments may not be possible from
every origin to every destination. The shipping costs per unit may be provided only for
the usable origin-destination pairs, so that it is desirable to index the costs and the variables
only over these pairs. In AMPL terms, we want the set `LINKS` defined above to
contain just a subset of pairs that are given in the data, rather than all pairs from `ORIG`
and `DEST`.

It is not sufficient to declare set `LINKS`, because that declares only a set of single
members. At a minimum, we need to say
```
set LINKS dimen 2;
```
to indicate that the data must consist of members of "dimension" two — that is, pairs.
Better yet, we can say that `LINKS` is a subset of the set of all pairs from `ORIG` and
`DEST`:
```
set LINKS within {ORIG,DEST};
```
This has the advantage of making the model's intent clearer; it also helps catch errors in
the data. The subsequent declarations of parameter cost, variable `Trans`, and the
objective function remain the same as they are in [Figure 6-1](../06/6_1_sets_of_ordered_pairs.ipynb#fig-6-1). But the components
`cost[i,j]` and `Trans[i,j]` will now be defined only for those pairs given in the
data as members of `LINKS`, and the expression
```
sum {(i,j) in LINKS} cost[i,j] * Trans[i,j]
```
will represent a sum over the specified pairs only.

How are the constraints written? In the original transportation model, the supply limit
constraint was:
```
subject to Supply {i in ORIG}:
  sum {j in DEST} Trans[i,j] = supply[i];
```
This does not work when LINKS is a subset of pairs, because for each `i in ORIG` it tries
to sum `Trans[i,j]` over every `j in DEST`, while `Trans[i,j]` is defined only for
pairs `(i,j) in LINKS`. If we try it, we get an error message like this:
```
error processing constraint Supply['GARY']:
       invalid subscript Trans['GARY','FRA']
```
What we want to say is that for each origin i, the sum should be over all destinations `j`
such that `(i,j)` is an allowed link. This statement can be transcribed directly to AMPL,
by adding a condition to the indexing expression after sum:
```
subject to Supply {i in ORIG}:
  sum {j in DEST: (i,j) in LINKS} Trans[i,j] = supply[i];
```
Rather than requiring this somewhat awkward form, however, AMPL lets us drop the `j in DEST`
from the indexing expression to produce the following more concise constraint:
```
subject to Supply {i in ORIG}:
  sum {(i,j) in LINKS} Trans[i,j] = supply[i];
```
Because `{(i,j) in LINKS}` appears in a context where `i` has already been defined,
AMPL interprets this indexing expression as the set of all `j` such that `(i,j) is in
LINKS`. The demand constraint is handled similarly, and the entire revised version of the
model is shown in [Figure 6-2a](../06/6_2_subsets_and_slices_of_ordered_pairs.ipynb#fig-6-2a). A small representative collection of data for this model is
shown in [Figure 6-2b](../06/6_2_subsets_and_slices_of_ordered_pairs.ipynb#fig-6-2b); AMPL offers a variety of convenient ways to specify the membership
of compound sets and the data indexed over them, as explained in Chapter 9. xTODO

You can see from [Figure 6-2a](../06/6_2_subsets_and_slices_of_ordered_pairs.ipynb#fig-6-2a) that the indexing expression
```
{(i,j) in LINKS}
```
means something different in each of the three places where it appears. Its membership
can be understood in terms of a table like this:
```
	    FRA    DET    LAN   WIN    STL    FRE    LAF
GARY             x      x            x             x
CLEV      x      x      x      x     x             x
PITT      x                    x     x      x
```
The rows represent origins and the columns destinations, while each pair in the set is
marked by an x. A table for `{ORIG,DEST}` would be completely filled in with x's,
while the table shown depicts `{LINKS}` for the "sparse" subset of pairs defined by the
data in [Figure 6-2b](../06/6_2_subsets_and_slices_of_ordered_pairs.ipynb#fig-6-2b).

<a id='fig-6-2a'><center><b>Figure 6-2a:</b> Transportation model with selected pairs (transp3.mod).</center></a>

In [5]:
%%writefile transp3.mod

set ORIG;                       # origins
set DEST;                       # destinations
set LINKS within {ORIG,DEST};
param supply {ORIG} >= 0;       # amounts available at origins
param demand {DEST} >= 0;       # amounts required at destinations
       check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];
param cost {LINKS} >= 0;        # shipment costs per unit
var Trans {LINKS} >= 0;         # units to be shipped
minimize Total_Cost:
  sum {(i,j) in LINKS} cost[i,j] * Trans[i,j];
subject to Supply {i in ORIG}:
  sum {(i,j) in LINKS} Trans[i,j] = supply[i];
subject to Demand {j in DEST}:
  sum {(i,j) in LINKS} Trans[i,j] = demand[j];

Overwriting transp3.mod


<a id='fig-6-2b'><center><b>Figure 6-2b:</b> Data for transportation model.</center></a>

In [6]:
ORIG = ["GARY", "CLEV", "PITT"]
DEST = ["FRA", "DET", "LAN", "WIN", "STL", "FRE", "LAF"]

supply = {"GARY": 1400, "CLEV": 2600, "PITT": 2900}
demand = {"FRA": 900, "DET": 1200, "LAN": 600, "WIN": 400, "STL": 1700, "FRE": 1100, "LAF": 1000}

cost = {
    ("GARY", "DET"): 14, ("GARY", "LAN"): 11, ("GARY", "STL"): 16, ("GARY", "LAF"): 8,
    ("CLEV", "FRA"): 27, ("CLEV", "DET"): 9,  ("CLEV", "LAN"): 12, ("CLEV", "WIN"): 9,
    ("CLEV", "STL"): 26, ("CLEV", "LAF"): 17,
    ("PITT", "FRA"): 24, ("PITT", "WIN"): 13, ("PITT", "STL"): 28, ("PITT", "FRE"): 99
}

In [16]:
from amplpy import AMPL
ampl = AMPL()
ampl.read('transp3.mod')

ampl.set["ORIG"] = ORIG
ampl.set["DEST"] = DEST
# LINKS are implicit in the "cost" dictionary
ampl.set["LINKS"] = list(cost.keys())
ampl.param["supply"] = supply
ampl.param["demand"] = demand
ampl.param["cost"] = cost

ampl.solve(solver='highs')

Trans_sol = ampl.get_solution()
print('Total transportation cost: ', ampl.obj['Total_Cost'].value())
for link in Trans_sol.items():
    print(link[0], '=', link[1])

HiGHS 1.11.0: optimal solution; objective 201700
2 simplex iterations
0 barrier iterations
Total transportation cost:  201700.0
Trans['GARY','STL'] = 800
Trans['GARY','LAF'] = 600
Trans['CLEV','DET'] = 1200
Trans['CLEV','LAN'] = 600
Trans['CLEV','WIN'] = 400
Trans['CLEV','LAF'] = 400
Trans['PITT','FRA'] = 900
Trans['PITT','STL'] = 900
Trans['PITT','FRE'] = 1100


At a point where `i` and `j` are not currently defined, such as in the objective 
```
minimize Total_Cost:
  sum {(i,j) in LINKS} cost[i,j] * Trans[i,j];
```
the indexing expression `{(i,j) in LINKS}` represents all the pairs in this table. But at
a point where `i` has already been defined, such as in the `Supply` constraint
```
subject to Supply {i in ORIG}:
 sum {(i,j) in LINKS} Trans[i,j] = supply[i];
```
the expression `{(i,j) in LINKS}` is associated with just the row of the table corresponding
to `i`. You can think of it as taking a one-dimensional "slice" through the table
in the row corresponding to the already-defined first component. Although in this case
the first component is a previously defined dummy index, the same convention applies
when the first component is any expression that can be evaluated to a valid set object; we
could write
```
{("GARY",j) in LINKS}
```
for example, to represent the pairs in the first row of the table.

Similarly, where `j` has already been defined, such as in the `Demand` constraint
```
subject to Demand {j in DEST}:
   sum {(i,j) in LINKS} Trans[i,j] = demand[j];
```
the expression `{(i,j) in LINKS}` selects pairs from the column of the table corresponding
to `j`. Pairs in the third column of the table could be specified by `{(i,"LAN") in LINKS}`.
